<a href="https://colab.research.google.com/github/Ashis-Palai/Cancer_Information_RAG_GenAI/blob/main/GenAI_Cancer_Information_Powered_by_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DATA EXTRACTION:**



## **Sources**

* **Cervical , Breast & Oral Cancers:**


> ```
https://tmc.gov.in/ncg/docs/PDF/DraftGuidelines/Preventive/3_%20NCG_INDIA_Rev_Preventive%20Oncology_Primary_Care.pdf
```


* **Ovarian_Cancer:**

> ```
https://main.icmr.nic.in/sites/default/files/guidelines/Ovarian_Cancer.pdf
```


* **Generic Information About Cancer**

>```
https://www.mayoclinic.org/diseases-conditions/cancer/diagnosis-treatment/drc-20370594
```



* **Breast Cancer:**

>```
https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection.html
```






In [1]:
!curl -O "https://raw.githubusercontent.com/Ashis-Palai/Cancer_Information_RAG_GenAI/main/helper.py"
!curl -O "https://raw.githubusercontent.com/Ashis-Palai/Cancer_Information_RAG_GenAI/main/requirements.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3168  100  3168    0     0  24318      0 --:--:-- --:--:-- --:--:-- 24369
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   130  100   130    0     0    903      0 --:--:-- --:--:-- --:--:--   909


In [2]:
all_url = [ 'https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection.html',
'https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/american-cancer-society-recommendations-for-the-early-detection-of-breast-cancer.html',
'https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/mammograms.html',
'https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/breast-ultrasound.html',
'https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/breast-mri-scans.html',
'https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/breast-cancer-signs-and-symptoms.html',
'https://tmc.gov.in/ncg/docs/PDF/DraftGuidelines/Preventive/3_%20NCG_INDIA_Rev_Preventive%20Oncology_Primary_Care.pdf',
'https://main.icmr.nic.in/sites/default/files/guidelines/Ovarian_Cancer.pdf'
            ]

In [3]:
from helper import extract_headings_and_content , word_wrap
for i, url in enumerate(all_url, start=1):
    result_message = extract_headings_and_content(url, i)
    print(result_message)

Extraction and saving successful for https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection.html. File saved as data/html_data/html_file_1.html
Extraction and saving successful for https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/american-cancer-society-recommendations-for-the-early-detection-of-breast-cancer.html. File saved as data/html_data/html_file_2.html
Extraction and saving successful for https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/mammograms.html. File saved as data/html_data/html_file_3.html
Extraction and saving successful for https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/breast-ultrasound.html. File saved as data/html_data/html_file_4.html
Extraction and saving successful for https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/breast-mri-scans.html. File saved as data/html_data/html_file_5.htm

# **DATA TRANSFORMATION USING LANGCHAIN**

In [5]:
from langchain_community.document_loaders import PyPDFLoader , UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter , SentenceTransformersTokenTextSplitter
import glob

In [6]:


base_path = '/content/data'

# List all HTML files
html_files = glob.glob(f"{base_path}/html*/*.html")

# List all PDF files
pdf_files = glob.glob(f"{base_path}/pdf*/*.pdf")


all_results = [PyPDFLoader(file).load() if file.endswith(".pdf") else UnstructuredHTMLLoader(file).load() for file in pdf_files + html_files]



In [7]:
total_pages = sum([len(i) for i in all_results ])
total_pages

107

In [8]:
all_text_data = [j.page_content.strip() for i in all_results for j  in i]

In [9]:
len(all_text_data)

107

In [10]:
all_text_data = [ i for i in all_text_data if i] # To exclude the blank pages

In [11]:
len(all_text_data)

102

In [12]:
splitter_1 =  RecursiveCharacterTextSplitter(
    separators= ['\n\n\n\n\n','\n\n\n\n','\n\n\n','\n\n','\n','.',',',' '],
    chunk_size = 1000 ,chunk_overlap = 100)

splitter_2 = SentenceTransformersTokenTextSplitter(tokens_per_chunk=256,chunk_overlap=10)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
prefinal_data = splitter_1.split_text('\n'.join(all_text_data))
len(prefinal_data)

313

In [14]:
final_data = []
[final_data.extend(splitter_2.split_text(text)) for text in prefinal_data]

len(final_data)

385

In [15]:
prefinal_data[0]

'Coordinated by:\nDivision of Non Communicable Diseases\nIndian Council of Medical Research  \nAnsari Nagar, New Delhi – 110029\n2019CONSENSUS DOCUMENT FOR\nMANAGEMENT OF\nEPITHELIAL OVARIAN CANCER\nPrepared as an outcome of ICMR Subcommittee on\nEpithelial Ovarian Cancer\nProf. Balram Bhargava  \nSecretary,  \nDepartment of Health Research  \nand Director General, ICMR\nPublished in 2019\nHead (Publication & Information) : Dr. Neeraj Tandon\nCompiled & Edited by: Dr. Hemant Tongaonkar & Dr. Tanvir Kaur\nProduction Controller : Mr. JN Mathur \nPublished by the Division of Publication and Information on behalf of the Secretary DHR & DG, ICMR,  \nNew Delhi. \nDesigned & Printed at M/s Royal Offset Printers, A-89/1, Naraina Industrial Area, Phase-I, New Delhi-110028  \nMobile: 9811622258Disclaimer\nThis consensus document represents the current thinking of experts on the topic based on available \nevidence. This has been developed by national experts in the field and does not in any way b

In [16]:
final_data[0]

'coordinated by : division of non communicable diseases indian council of medical research ansari nagar, new delhi – 110029 2019consensus document for management of epithelial ovarian cancer prepared as an outcome of icmr subcommittee on epithelial ovarian cancer prof. balram bhargava secretary, department of health research and director general, icmr published in 2019 head ( publication & information ) : dr. neeraj tandon compiled & edited by : dr. hemant tongaonkar & dr. tanvir kaur production controller : mr. jn mathur published by the division of publication and information on behalf of the secretary dhr & dg, icmr, new delhi. designed & printed at m / s royal offset printers, a - 89 / 1, naraina industrial area, phase - i, new delhi - 110028 mobile : 9811622258disclaimer this consensus document represents the current thinking of experts on the topic based on available evidence. this has been developed by national experts in the field and does not in any way bind'

# **DATA LOADING**

## **Embedding : GooglePalmEmbeddings**

In [ ]:
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_community.embeddings import GooglePalmEmbeddings
from google.colab import userdata

emb = GooglePalmEmbeddings(model_name='models/embedding-gecko-001',google_api_key=GOOGLE_API_KEY)
print(f"Out put dimension: {len(emb.embed_query('Hi How are you?'))}")

## **Data Base : Pinecone Vector DB**

In [ ]:
from pinecone import Pinecone, ServerlessSpec, PodSpec


p = Pinecone(api_key= PINECONE_API_KEY)

In [ ]:
p.create_index('cancer-retrieval',
              dimension=768,
              metric='cosine',
              spec = PodSpec(environment='gcp-starter',
                             pod_type='starter',
                             pods = 1))
index =  p.Index('cancer-retrieval')

In [ ]:
index =  p.Index('cancer-retrieval')

In [ ]:
index.describe_index_stats()